In [2]:
import cv2
import numpy as np
import time

In [3]:
def create_equirectangular_map(input_w, input_h, output_sqr, displacement, direction='z'):
    row_indices, col_indices = np.meshgrid(np.arange(output_sqr), np.arange(output_sqr))
    x = y = z = np.zeros_like(row_indices, dtype=float)
    
    if direction == 'z':
        x = row_indices - output_sqr / 2.0
        y = col_indices - output_sqr / 2.0
        z += displacement
    elif direction == 'x':
        y = row_indices - output_sqr / 2.0
        z = col_indices - output_sqr / 2.0
        x += displacement
    elif direction == 'y':
        z = row_indices - output_sqr / 2.0
        x = col_indices - output_sqr / 2.0
        y += displacement

    rho = np.sqrt(x**2 + y**2 + z**2)
    norm_theta = np.arctan2(y, x) / (2 * np.pi)
    norm_phi = (np.pi - np.arccos(z / rho)) / np.pi
    ix = norm_theta * input_w
    iy = norm_phi * input_h

    ix = ix % input_w
    iy = iy % input_h

    return ix, iy


def create_cube_map(
    bottom_img, top_img, front_img, back_img, left_img, right_img, output_sqr
):
    output_w = output_sqr * 4
    output_h = output_sqr * 3
    cube_map_img = np.zeros((output_h, output_w, 3), dtype=np.uint8)

    cube_map_img[output_sqr * 2 : output_h, output_sqr : output_sqr * 2] = bottom_img
    cube_map_img[0:output_sqr, output_sqr : output_sqr * 2] = top_img
    cube_map_img[output_sqr : output_sqr * 2, output_sqr : output_sqr * 2] = front_img
    cube_map_img[output_sqr : output_sqr * 2, output_sqr * 3 : output_w] = back_img
    cube_map_img[output_sqr : output_sqr * 2, 0:output_sqr] = left_img
    cube_map_img[
        output_sqr : output_sqr * 2, output_sqr * 2 : output_sqr * 3
    ] = right_img

    return cube_map_img


def main(image_path, output_sqr=256, margin=0):
    img = cv2.imread(image_path)
    normalized_f = 2.0
    displacement = output_sqr / normalized_f
    expanded_output_sqr = output_sqr + margin * 2
    input_h, input_w = img.shape[0], img.shape[1]
    

    time_b = time.perf_counter()
    bottom_map_x, bottom_map_y = create_equirectangular_map(
        input_w, input_h, expanded_output_sqr, displacement, 'z'
    )
    top_map_x, top_map_y = create_equirectangular_map(
        input_w, input_h, expanded_output_sqr, -displacement, 'z'
    )
    front_map_x, front_map_y = create_equirectangular_map(
        input_w, input_h, expanded_output_sqr, -displacement, 'x'
    )
    back_map_x, back_map_y = create_equirectangular_map(
        input_w, input_h, expanded_output_sqr, displacement, 'x'
    )
    left_map_x, left_map_y = create_equirectangular_map(
        input_w, input_h, expanded_output_sqr, -displacement, 'y'
    )
    right_map_x, right_map_y = create_equirectangular_map(
        input_w, input_h, expanded_output_sqr, displacement, 'y'
    )
    bottom_img = cv2.remap(
        img,
        bottom_map_x.astype("float32"),
        bottom_map_y.astype("float32"),
        cv2.INTER_CUBIC,
    )
    bottom_img = cv2.rotate(bottom_img, cv2.ROTATE_90_CLOCKWISE)
    
    top_img = cv2.remap(
        img, top_map_x.astype("float32"), top_map_y.astype("float32"), cv2.INTER_CUBIC
    )
    top_img = cv2.flip(top_img, 1)
    top_img = cv2.rotate(top_img, cv2.ROTATE_90_CLOCKWISE)

    front_img = cv2.remap(
        img,
        front_map_x.astype("float32"),
        front_map_y.astype("float32"),
        cv2.INTER_CUBIC,
    )
    front_img = cv2.flip(front_img, 1)


    back_img = cv2.remap(
        img, back_map_x.astype("float32"), back_map_y.astype("float32"), cv2.INTER_CUBIC
    )
    


    left_img = cv2.remap(
        img, left_map_x.astype("float32"), left_map_y.astype("float32"), cv2.INTER_CUBIC
    )
    left_img = cv2.flip(left_img, 1)
    left_img = cv2.rotate(left_img, cv2.ROTATE_90_COUNTERCLOCKWISE)

    right_img = cv2.remap(
        img,
        right_map_x.astype("float32"),
        right_map_y.astype("float32"),
        cv2.INTER_CUBIC,
    )
    right_img = cv2.rotate(right_img, cv2.ROTATE_90_CLOCKWISE)
    time_a = time.perf_counter()

    print(time_a-time_b)

    cv2.imwrite("./output/bottom.png", bottom_img)
    cv2.imwrite("./output/top.png", top_img)
    cv2.imwrite("./output/front.png", front_img)
    cv2.imwrite("./output/back.png", back_img)
    cv2.imwrite("./output/right.png", right_img)
    cv2.imwrite("./output/left.png", left_img)
    

    # Create cube map image
    #if margin == 0:
    cube_map_img = create_cube_map(
        bottom_img, top_img, front_img, back_img, right_img, left_img, output_sqr+margin*2
    )
    cv2.imwrite("./output/cube_map.png", cube_map_img)


if __name__ == "__main__":
    image_path = "./data/data_100/Room/0/O.png"
    main(image_path)

0.023364280999885523
